In [65]:
import pandas as pd
import numpy as np
import re
import jieba
from sklearn.model_selection import train_test_split
import gensim

In [88]:
train = pd.read_csv("train.csv",delimiter="\t")
test = pd.read_csv("test.csv",delimiter="\t")

In [89]:
def stopwordslist():
    stopwords = [line.strip() for line in open('cn_stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

def removepre(sentence):
    pattern = re.compile("[^a-zA-Z0-9\u4E00-\u9FA5]")
    sentence = pattern.sub("",sentence)
    sentence = sentence.replace("#","")
    sentence = sentence.replace("【","")
    sentence = sentence.replace("】","")
    return sentence


def removenumber(sentence):
    pattern = re.compile("[\d+]")
    sentence = pattern.sub("",sentence)
    return sentence
    
        
def seg_depart(sentence):
     # 对文档中的每一行进行中文分词
    sentence_depart = jieba.cut(sentence.strip())
     # 创建一个停用词列表
    stopwords = stopwordslist()
    # 输出结果为outstr
    outstr = ''
   # 去停用词
    for word in sentence_depart:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr  

In [90]:
train["cut"] = train["text"].apply(removepre)
test["cut"] = test["text"].apply(removepre)

train["cut"] = train["cut"].apply(removenumber)
test["cut"] = train["cut"].apply(removenumber)

train["cut"] = train["cut"].apply(seg_depart)
test["cut"] = train["cut"].apply(seg_depart)



In [91]:
train_set = [i for i in train["cut"]]
test_set = [i for i in test["cut"]]

In [92]:
training = train_set+test_set

In [114]:
len(training)

3000

In [124]:
train.to_csv('dataset/training.csv',index=0)

In [123]:
test.to_csv('dataset/testing.csv',index=0)

In [126]:
pd.read_csv("dataset/testing.csv")

,target,text,cut
0,IphoneSE,讲真，对iphoneSE很心动，但是又很期待iphone7，心塞,刚 回家 几天 迫不及待 赶到 小舅 家 看着 乖巧 懂...
1,春节放鞭炮,传统春节来临传统的拜神，放鞭炮，烧纸，热闹呢,俄罗斯 流氓
2,俄罗斯在叙利亚的反恐行动,俄罗斯在战争状态下的紧急动员能力，这不是土耳其可以低估的。,春节 放鞭炮 中央气象台 首次 发布 烟花爆竹 燃放 气象 ...
3,深圳禁摩限电,珠海要是有这么高强度，市区就不会有那么多摩托车横冲直撞了,iPhoneSE 貌似 摄像头 外突 普天同庆
4,深圳禁摩限电,#深圳禁摩限电# 早该整了，快递开电车，真把马路当成他们家开的一样...不过他们工作压力大，...,千万 人口 级 城市 北京 原住民 庆祝 第一 节日 春...
...,...,...,...
595,春节放鞭炮,发表了博文 《小年印象》 - 不知不觉，小年到了，俗称“祭灶”，即希望灶神“上天言好事，下地...,霉 奸 乌克兰 不会 战乱 中东 不会 大乱 伊死烂国 ...
596,俄罗斯在叙利亚的反恐行动,这个土耳其一沾上他美爹的光就强硬起来了，不想想自己是个啥德行？自不量力！,皇恩浩荡 终于 开放 二胎 政策 记得 小时候 经常 看到 ...
597,春节放鞭炮,今年上海过节放鞭炮的现象大为减少，空气质量也比较好，这次政府管制得很好，不断发劝告短消息给市...,苹果公司 宣布 美国 时间 月 日 发布 新 产品 包括...
598,IphoneSE,喜欢小屏幕的苹果，现在还是4s，好吧，其实还有是因为穷,网上 曝光 图 看起来 不错 不太 喜欢 大屏 手机 兜


In [38]:
a = [i.split(" ") for i in training]

In [94]:
from gensim.models import word2vec
import multiprocessing
 
def train_wordVectors(sentences, embedding_size = 50, window = 5, min_count = 1):
    '''
    :param sentences: sentences可以是LineSentence或者PathLineSentences读取的文件对象，也可以是
                    The `sentences` iterable can be simply a list of lists of tokens,如lists=[['我','是','中国','人'],['我','的','家乡','在','广东']]
    :param embedding_size: 词嵌入大小
    :param window: 窗口
    :param min_count:Ignores all words with total frequency lower than this.
    :return: w2vModel
    '''
    w2vModel = word2vec.Word2Vec(sentences, size=embedding_size, window=window, min_count=min_count,workers=multiprocessing.cpu_count())
    return w2vModel
 
def save_wordVectors(w2vModel,word2vec_path):
    w2vModel.save(word2vec_path)
 
def load_wordVectors(word2vec_path):
    w2vModel = word2vec.Word2Vec.load(word2vec_path)
    return w2vModel
 
if __name__=='__main__':
 
    word2vec_path='word2Vec.model'
    model2=train_wordVectors(a, embedding_size=50, window=5, min_count=1)
    save_wordVectors(model2,word2vec_path)
    model2=load_wordVectors(word2vec_path)
    print(model2.wv.similarity("深圳","俄罗斯"))


0.91149235


In [70]:
model = gensim.models.KeyedVectors.load_word2vec_format(r"word embedding/cn.skipgram.bin",binary=True,unicode_errors='ignore')

In [73]:
model.wv.similarity("深圳","俄罗斯")

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


0.1635503

In [95]:
def to_review_vector(review):
    

    #words = nltk.word_tokenize(review)
    word_vec = np.zeros((1,50))
    for word in review:
        #word_vec = np.zeros((1,300))
        if word in model2:
            word_vec += np.array([model2[word]])
    #print (word_vec.mean(axis = 0))
    return pd.Series(word_vec.mean(axis = 0))

train_data_features_0 = train["cut"].apply(to_review_vector)
train_data_features_0.head()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
D:\anaconda\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,4.583189,2.130372,-26.202713,-8.923727,6.484468,18.861293,12.946058,-2.785165,-5.749008,-3.827543,...,-13.009636,0.645828,-14.657746,-10.470855,-21.277053,-1.430176,9.096585,5.078261,-3.410602,-11.019945
1,0.195311,0.079864,-1.025793,-0.339263,0.254341,0.751464,0.514491,-0.109752,-0.205636,-0.153152,...,-0.500704,0.045291,-0.592562,-0.421232,-0.867570,-0.049362,0.374922,0.193969,-0.146967,-0.438258
2,5.596611,2.724850,-33.270599,-11.268319,8.623322,24.379468,16.169671,-3.566423,-7.891737,-4.962621,...,-16.314008,1.214932,-18.347073,-13.366442,-27.243764,-1.308892,11.154958,6.673887,-4.547123,-13.739221
3,0.567001,0.222205,-3.164547,-1.068910,0.813371,2.286470,1.559399,-0.290324,-0.677558,-0.488348,...,-1.603483,0.132228,-1.799794,-1.308537,-2.643936,-0.166808,1.067169,0.569538,-0.409830,-1.292601
4,5.491954,2.807478,-30.797740,-10.547736,7.676270,22.303603,15.319292,-3.538073,-6.826188,-4.595487,...,-15.368514,0.737322,-17.203267,-12.231512,-25.049349,-1.436152,10.615528,5.948612,-3.978248,-13.063545


In [96]:
train["stance"].unique()

array(['FAVOR', 'AGAINST', 'NONE'], dtype=object)

In [97]:
train["stance"]=train["stance"].map({"FAVOR":0,"AGAINST":1,"NONE":2})

In [98]:
X = train.drop("stance",axis=1)
Y = train["stance"]

In [99]:
x_train,x_test,y_train,y_test = train_test_split(train_data_features_0,Y,test_size=0.1,random_state=1)

In [100]:
x_train.shape

(2160, 50)

In [101]:
y_train.shape

(2160,)

In [102]:
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression()
LR_model = LR_model.fit(x_train, y_train)
y_pred = LR_model.predict(x_test)

D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [103]:
np.mean(y_pred == y_test)

0.5125

In [104]:
from xgboost import XGBClassifier

my_model = XGBClassifier()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(x_train, y_train)
predictions = my_model.predict(x_test)

np.mean(predictions == y_test)

0.43333333333333335

In [109]:

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier

In [112]:
model = OneVsRestClassifier(GradientBoostingClassifier(learning_rate=0.1,n_estimators=1000,max_depth=7))
clt = model.fit(x_train, y_train)

In [113]:
clt.score(x_test,y_test)

0.4708333333333333